# MAUDE 데이터 탐색

## 라이브러리 불러오기

In [1]:
from typing import Tuple, List
import polars as pl
from pprint import pprint
import sys
from pathlib import Path
import psutil

# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
from code.loading import DataLoader

In [2]:
# 시스템의 논리적 CPU 코어 수를 기반으로 워커 수 결정
optimal_workers = psutil.cpu_count(logical=True)

print(f"사용할 워커 수: {optimal_workers}")

# 시스템의 전체 메모리 정보를 가져옴
mem = psutil.virtual_memory()

print(f"총 메모리: {mem.total / (1024**3):.2f} GB")
print(f"사용 가능 메모리: {mem.available / (1024**3):.2f} GB")
print(f"사용 중인 메모리: {mem.used / (1024**3):.2f} GB")
print(f"메모리 사용률: {mem.percent}%")

사용할 워커 수: 64
총 메모리: 125.14 GB
사용 가능 메모리: 112.51 GB
사용 중인 메모리: 12.63 GB
메모리 사용률: 10.1%


## 원본 데이터 불러오기

In [3]:

loader = DataLoader(
    start=2020,
    end=2025,
    output_file = DATA_DIR / 'output.parquet',
    max_workers=4
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
df = loader.load(adapter=adapter, **polars_kwargs)
df


📖 /home/dataiku/eric/Sparta/Project3/data/output.parquet 로딩 중... (adapter=polars)


In [4]:
# 전역 변수
BASE_COLS = [
    'mdr_report_key', 'adverse_event_flag', 'product_problems',
    'product_problem_flag', 'date_of_event', 'date_report', 
    'date_received', 'device_date_of_manufacturer', 'event_type',
    'previous_use_code', 'single_use_flag', 'report_source_code',
    'reprocessed_and_reused_flag', 'date_facility_aware', 'report_date',
    'report_to_fda', 'date_report_to_fda', 'report_to_manufacturer',
    'date_report_to_manufacturer', 'event_location', 
    'date_manufacturer_received', 'manufacturer_link_flag',
    'date_added', 'date_changed', 'pma_pmn_number',
    'suppl_dates_fda_received', 'suppl_dates_mfr_received'
]

# device_* 특정 컬럼들
DEVICE_PATTERNS = [
    '_brand_name', '_udi_di', '_device_report_product_code', 
    '_model_number', '_expiration_date_of_device', '_device_age_text',
    '_device_operator', '_implant_flag', '_manufacturer_d_name', 
    '_openfda_device_class', '_openfda_device_name', '_generic_name'
]

# patient_* 특정 컬럼들
PATIENT_PATTERNS = [
    '_patient_sequence_number', '_patient_age', '_patient_sex',
    '_patient_weight', '_patient_race', '_patient_problems',
    '_sequence_number_outcome'
]

# mdr_text_* 특정 컬럼들
MDR_TEXT_PATTERNS = ['_text', '_text_type_code']

## 사용할 컬럼만 1차 추출

In [5]:
def get_use_cols(
    df: pl.LazyFrame,
    base_cols: List[str] = BASE_COLS,
    device_patterns: List[str] = DEVICE_PATTERNS,
    patient_patterns: List[str] = PATIENT_PATTERNS,
    mdr_text_patterns: List[str] = MDR_TEXT_PATTERNS
) -> Tuple[List[str], List[str], List[str], List[str]]:
    
    total_cols = df.collect_schema().names()
    device_cols = [col for col in total_cols if col.startswith('device_') and 
                any(pattern in col for pattern in device_patterns)]
    patient_cols = [col for col in total_cols if col.startswith('patient_') and 
                    any(pattern in col for pattern in patient_patterns)]
    mdr_text_cols = [col for col in total_cols if col.startswith('mdr_text_') and 
                    any(col.endswith(pattern) for pattern in mdr_text_patterns)]

    # 합치기
    analysis_cols = base_cols + device_cols + patient_cols + mdr_text_cols
    analysis_cols = sorted(list(set(analysis_cols)), reverse=True)

    print(f"총 컬럼: {len(analysis_cols)}개")
    print(f"  - 기본 컬럼: {len(base_cols)}개")
    print(f"  - device_*: {len(device_cols)}개")
    print(f"  - patient_*: {len(patient_cols)}개")
    print(f"  - mdr_text_*: {len(mdr_text_cols)}개")

    return device_cols, patient_cols, mdr_text_cols, analysis_cols

device_cols, patient_cols, mdr_text_cols, analysis_cols = get_use_cols(df)

총 컬럼: 835개
  - 기본 컬럼: 27개
  - device_*: 622개
  - patient_*: 26개
  - mdr_text_*: 160개


## 타입 확인

In [6]:
target_col = 'adverse_event_flag'
df.select(
    pl.col(target_col).n_unique().alias(f'unique_{target_col}')
).collect()

df.select(
    pl.col(target_col).unique().sort().head(100).alias(f'head_{target_col}'),
    pl.col(target_col).unique().sort().tail(100).alias(f'tail_{target_col}'),
).collect().to_pandas()

,head_adverse_event_flag,tail_adverse_event_flag
0,None,None
1,N,N
2,Y,Y


In [7]:
def overview_col(df: pl.LazyFrame, col: str):
    nunique = df.select(
        pl.col(col).n_unique().alias(f'unique_{col}')
    ).collect().item()
    
    print(f'{col}의 고유 개수: {nunique}')
    
    unique_df = df.select(
        pl.col(col).unique().sort().head(100).alias(f'head_{col}'),
        pl.col(col).unique().sort().tail(100).alias(f'tail_{col}'),
    ).collect().to_pandas()
    
    display(unique_df)

In [8]:
# for col in BASE_COLS:
#     overview_col(df, col)

In [9]:
first_device_cols = [col for col in device_cols if col[7] == '0']

# for col in first_device_cols:
#     overview_col(df, col)

In [10]:
first_patient_cols = [col for col in patient_cols if col[8] == '0']

# for col in first_patient_cols:
#     overview_col(df, col)

In [11]:
first_mdr_cols = [col for col in mdr_text_cols if '_0_' in col]

for col in first_mdr_cols:
    overview_col(df, col)

mdr_text_0_text의 고유 개수: 4228768


,head_mdr_text_0_text,tail_mdr_text_0_text
0,None,¿THE FAILURES LEADING TO MOTOR STALL (ERROR CO...
1,(B)(4).,¿THE FAILURES LEADING TO MOTOR STALL (ERROR CO...
2,(B)(4). CANCEL AND CLOSE. S/W VERSION 4.11J. ...,¿THE FAILURES LEADING TO MOTOR STALL (ERROR CO...
3,(B)(4). CURRENTLY IT IS UNKNOWN WHETHER OR NO...,¿THE FOLLOWING FIELDS HAVE BEEN UPDATED WITH C...
4,(B)(4). CURRENTLY IT IS UNKNOWN WHETHER OR NO...,¿THE FREE STYLE LIBRE 3 MEASURED MY HUSBANDS B...
...,...,...
95,"""...VENTILATOR WAS NOTED TO HAVE FLICKED INTO ...","ÖKÇESIZ, I., DÖNMEZ, H., VURAL, A., KARABIYIK,..."
96,"""032227-060322-002-C. (B)(4). CURRENTLY IT IS ...","ÖZDEMIR-VAN BRUNSCHOT D, HARRICH F, TEVS M, HO..."
97,"""04/18/24 EVALUATION TECH: TL EMN HP; CABLE, C...","ÖZDEMIR-VAN BRUNSCHOT D, HARRICH F, TEVS M, HO..."
98,"""1 PORT OPERATED ON 1 CHILD. STARTED BLEEDING ...","ï»¿Unique Complaint ID Number,Event Date,Date ..."


mdr_text_0_text_type_code의 고유 개수: 3


,head_mdr_text_0_text_type_code,tail_mdr_text_0_text_type_code
0,None,None
1,Additional Manufacturer Narrative,Additional Manufacturer Narrative
2,Description of Event or Problem,Description of Event or Problem


### 타입별 컬럼 모으기

In [12]:
date_cols = ['date_of_event', 'date_report', 'date_manufacturer_received', 'date_received', 'date_added', 'suppl_dates_fda_received', 'suppl_dates_mfr_received', 'device_date_of_manufacturer', 'device_0_expiration_date_of_device', 'date_changed']

In [13]:
def summarize_date_gaps(
    lf: pl.LazyFrame,
    date_col1: str = None,
    date_col2: str = None,
    fmt: str = "%Y%m%d",
) -> pl.LazyFrame:
    return (
        lf
        .select([date_col1, date_col2])
        .drop_nulls()
        .with_columns([
            pl.col(date_col1)
              .cast(pl.Utf8)
              .str.strptime(pl.Date, format=fmt, strict=False)
              .alias(date_col1),
            pl.col(date_col2)
              .cast(pl.Utf8)
              .str.strptime(pl.Date, format=fmt, strict=False)
              .alias(date_col2)
        ])
        .with_columns([
            (pl.col(date_col1).cast(pl.Int64) - pl.col(date_col2).cast(pl.Int64))
                .alias("days_date1_minus_date2"),
        ])
        .select([
            pl.col("days_date1_minus_date2").min().alias("min_date1_date2"),
            pl.col("days_date1_minus_date2").max().alias("max_date1_date2"),
            pl.col("days_date1_minus_date2").median().alias("med_date1_date2"),
            pl.col("days_date1_minus_date2").mean().alias("mean_date1_date2"),
        ])
    )


In [14]:
df.select(date_cols).head(10).collect().to_pandas()

,date_of_event,date_report,date_manufacturer_received,date_received,date_added,suppl_dates_fda_received,suppl_dates_mfr_received,device_date_of_manufacturer,device_0_expiration_date_of_device,date_changed
0,20200708,20200708,20200708,20200720,20200720,09/14/2020,09/03/2020,None,20210923,20230718
1,20200707,20200707,20200707,20200720,20200720,08/24/2020,08/21/2020,None,None,20230718
2,20200504,20200720,20200702,20200720,20200720,None,None,20191029,None,20200729
3,20200710,20200710,20200710,20200720,20200720,None,None,20150223,None,20200724
4,20200622,20200630,20200629,20200720,20200720,None,None,20200109,20210430,20200803
5,20200313,20200720,None,20200720,20200720,None,None,20150315,20200315,20200727
6,20200513,20200720,None,20200720,20200720,None,None,None,None,20200727
7,20200309,20200720,None,20200720,20200720,None,None,20181211,20231211,20200908
8,20200628,20200720,20200628,20200720,20200720,None,None,20190601,None,20200831
9,20200404,20200720,None,20200720,20200720,None,None,20181031,20231031,20200727


In [15]:
df.select(['device_date_of_manufacturer','device_0_udi_di', 'device_5_udi_di']).filter(
    pl.col('device_date_of_manufacturer').is_not_null() & \
    pl.col('device_5_udi_di').is_not_null()
).collect().to_pandas()

,device_date_of_manufacturer,device_0_udi_di,device_5_udi_di


In [16]:
def overview_col(df: pl.LazyFrame, col: str):
    nunique = df.select(
        pl.col(col).n_unique().alias(f'unique_{col}')
    ).collect().item()
    
    print(f'{col}의 고유 개수: {nunique}')
    
    unique_df = df.select(
        pl.col(col).unique().sort().head(100).alias(f'head_{col}'),
        pl.col(col).unique().sort().tail(100).alias(f'tail_{col}'),
    ).collect().to_pandas()
    
    display(unique_df)

In [17]:
df.select(analysis_cols).filter(
    pl.col('suppl_dates_mfr_received').is_not_null()
).head(10).collect().to_pandas()

,suppl_dates_mfr_received,suppl_dates_fda_received,single_use_flag,reprocessed_and_reused_flag,report_to_manufacturer,report_to_fda,report_source_code,report_date,product_problems,product_problem_flag,...,date_report_to_manufacturer,date_report_to_fda,date_report,date_received,date_of_event,date_manufacturer_received,date_facility_aware,date_changed,date_added,adverse_event_flag
0,09/03/2020,09/14/2020,None,N,None,*,Manufacturer report,None,"['Activation, Positioning or Separation Problem']",Y,...,None,None,20200708,20200720,20200708,20200708,None,20230718,20200720,N
1,08/21/2020,08/24/2020,None,N,None,*,Manufacturer report,None,['Loss of or Failure to Bond'],Y,...,None,None,20200707,20200720,20200707,20200707,None,20230718,20200720,Y
2,09/14/2020,09/21/2020,None,N,None,N,Manufacturer report,None,['Wireless Communication Problem'],Y,...,None,None,20200921,20200720,20200629,20200706,None,20230515,20200720,N
3,06/30/2020,08/11/2020,None,N,None,N,Manufacturer report,None,"['Unable to Obtain Readings', 'Failure of Devi...",Y,...,None,None,20200630,20200721,None,20200630,None,20230717,20200721,N
4,09/13/2020,09/21/2020,N,N,None,N,Manufacturer report,None,['No Device Output'],Y,...,None,None,20200921,20200721,20200625,20200626,None,20201010,20200721,N
5,07/22/2020,07/27/2020,N,N,None,*,Manufacturer report,None,['Adverse Event Without Identified Device or U...,N,...,None,None,20200726,20200721,20200707,20200707,None,20200729,20200721,Y
6,07/24/2020,08/21/2020,None,N,None,*,Manufacturer report,None,['Patient-Device Incompatibility'],N,...,None,None,20200821,20200722,20190101,20200622,None,20230517,20200722,Y
7,07/24/2020;11/13/2020,07/27/2020;11/18/2020,None,N,None,*,Manufacturer report,None,['Low Readings'],N,...,None,None,20201118,20200722,20200715,20200715,None,20230518,20200722,Y
8,09/01/2020,09/09/2020,N,N,None,Y,Manufacturer report,None,['Pumping Stopped'],Y,...,None,None,20200722,20200722,20200702,20200702,None,20200913,20200722,N
9,07/30/2020,08/11/2020,N,N,None,Y,Manufacturer report,None,"['Crack', 'Failure to Sense']",Y,...,None,None,20200722,20200722,20200630,20200630,None,20201023,20200722,N


In [18]:
date_col1 = 'date_added'
date_col2 = 'date_received'
date_gaps = summarize_date_gaps(df, date_col1, date_col2)
date_gaps.collect()

min_date1_date2,max_date1_date2,med_date1_date2,mean_date1_date2
i64,i64,f64,f64
-3647,1262,0.0,1.369733


In [19]:
df.select([date_col1, date_col2]).filter(
    pl.col(date_col1) < pl.col(date_col2)
).collect().to_pandas()

,date_added,date_received
0,20181018,20200710
1,20201220,20201221
2,20201212,20201213
3,20201220,20201221
4,20201220,20201221
5,20201220,20201221
6,20201220,20201221
7,20201220,20201221
8,20201220,20201221
9,20201220,20201221


In [20]:
def get_type_cols(
    df: pl.LazyFrame
):
    pass

## 제품군 확인

In [21]:
# 상표명, generic_name
cols = ['device_0_brand_name', 'device_0_generic_name', 'device_0_device_report_product_code', 'device_0_openfda_device_name']

df.select(pl.col(cols)).group_by('device_0_device_report_product_code').agg(
    pl.col('device_0_generic_name').n_unique().alias('unique_generic_name'),
    pl.col('device_0_openfda_device_name').n_unique().alias('unique_device_0_openfda_device_name'),
).collect().to_pandas().sort_values('unique_generic_name', ascending=False)

,device_0_device_report_product_code,unique_generic_name,unique_device_0_openfda_device_name
2359,JWH,2277,1
1460,DZE,1548,1
2133,LZO,1113,1
3147,LXH,1091,1
2131,FPA,992,1
...,...,...,...
5,LRH,1,1
3,KHQ,1,1
2204,MIP,1,1
2208,FPY,1,1


In [22]:
# 상표명, 등록번호
# cols = ['device_0_brand_name', 'device_0_generic_name']

df.select(pl.col(cols)).group_by('device_0_brand_name').agg(
    pl.col('device_0_generic_name').n_unique().alias('unique_generic_name')
).collect().to_pandas().sort_values('unique_generic_name', ascending=False)

,device_0_brand_name,unique_generic_name
145977,None,3574
7241,DJO SURGICAL,724
6849,ELEOS,402
78206,PENTAX,313
27774,UNKNOWN,284
...,...,...
76007,G6 TRANSMITTER CONTINUOUS BLOOD GLUCOSE MONITO...,1
76008,SAFESTEP HUBER NEEDLE 20G X 0.75IN WITH Y-SITE,1
76010,EVERFLO PORTABLE OXYGEN CONCENTRATOR,1
76011,AORTIC BIFURCATE,1


In [23]:
df.select(cols).filter(
    pl.col('device_0_device_report_product_code').str.contains('JWH')
).group_by('device_0_generic_name').agg(
    pl.col('device_0_generic_name').len().alias('cnt_generic_name'),
    pl.col('device_0_brand_name').mode().alias('mode_brand_name'),
    pl.col('device_0_openfda_device_name').mode().alias('mode_device_0_openfda_device_name'),
).collect().to_pandas().sort_values('cnt_generic_name', ascending=False)

,device_0_generic_name,cnt_generic_name,mode_brand_name,mode_device_0_openfda_device_name
1327,"PROSTHESIS, KNEE, PATELLOFEMOROTIBIAL, SEMI-CO...",10595,[SPECIFIC DEVICE NOT REPORTED],"[Prosthesis, Knee, Patellofemorotibial, Semi-C..."
403,"PROSTHESIS, KNEE",9274,[UNKNOWN ARTICULAR SURFACE],"[Prosthesis, Knee, Patellofemorotibial, Semi-C..."
830,"INSTRUMENT, KNEE",2653,[FEMORAL CR IMPACTOR PAD],"[Prosthesis, Knee, Patellofemorotibial, Semi-C..."
1393,KNEE TIBIAL INSERT,2436,[UNKNOWN KNEE TIBIAL INSERT],"[Prosthesis, Knee, Patellofemorotibial, Semi-C..."
303,KNEE TIBIAL TRAY,1924,[UNKNOWN KNEE TIBIAL TRAY],"[Prosthesis, Knee, Patellofemorotibial, Semi-C..."
...,...,...,...,...
1098,"CEMENTED EXTENSION STEM Ø13MM, 65MM",1,[GMK REVISION TOTAL KNEE SYSTEM],"[Prosthesis, Knee, Patellofemorotibial, Semi-C..."
1097,"FEMUR, REVISION EXPRT KNEE, SZ C LT",1,[None],"[Prosthesis, Knee, Patellofemorotibial, Semi-C..."
1092,"TIBIAL INSERT SC FIXED 14MM, SIZE 2",1,[GMK REVISION TOTAL KNEE SYSTEM],"[Prosthesis, Knee, Patellofemorotibial, Semi-C..."
1111,FIXED TIBIAL INSERT SEMICONSTRAINED S.3 / 14 MM,1,[GMK REVISION TOTAL KNEE SYSTEM],"[Prosthesis, Knee, Patellofemorotibial, Semi-C..."


## 결측 통일

In [24]:
def analyze_null_values(df, analysis_cols, verbose=True):
    """
    컬럼별 결측치 분석 함수
    
    Parameters:
    -----------
    df : pl.LazyFrame
        분석할 LazyFrame
    analysis_cols : list
        분석할 컬럼 리스트
    verbose : bool, default=True
        결과를 출력할지 여부
    
    Returns:
    --------
    tuple : (null_stats, null_df)
        null_stats: 결측치 통계 딕셔너리 리스트
        null_df: 결측치 통계를 담은 DataFrame
    """
    if verbose:
        print("\n=== 결측치 분석 ===")
    
    null_stats = []
    total_rows = df.select(pl.len()).collect().item()
    all_cols = df.collect_schema().names()
    
    for col in analysis_cols:
        if col in all_cols:
            null_count = df.select(pl.col(col).null_count()).collect().item()
            null_pct = null_count / total_rows * 100
            null_stats.append({
                'column': col,
                'null_count': null_count,
                'null_pct': null_pct
            })
            if verbose:
                print(f"{col:45s}: {null_count:>10,}개 ({null_pct:>5.2f}%)")
        else:
            if verbose:
                print(f"{col:45s}: 컬럼 없음")
    
    null_df = pl.DataFrame(null_stats).sort('null_pct', descending=True)
    return null_stats, null_df


# 함수 사용
null_stats, null_df = analyze_null_values(df, analysis_cols)


=== 결측치 분석 ===
suppl_dates_mfr_received                     :  9,447,860개 (69.25%)
suppl_dates_fda_received                     :  9,447,860개 (69.25%)
single_use_flag                              :  1,617,562개 (11.86%)
reprocessed_and_reused_flag                  :    572,761개 ( 4.20%)
report_to_manufacturer                       : 13,642,636개 (100.00%)
report_to_fda                                :  7,938,519개 (58.19%)
report_source_code                           :          0개 ( 0.00%)
report_date                                  : 12,843,328개 (94.14%)
product_problems                             :     35,140개 ( 0.26%)
product_problem_flag                         :      4,686개 ( 0.03%)
previous_use_code                            :  1,474,472개 (10.81%)
pma_pmn_number                               :  2,240,540개 (16.42%)
patient_3_sequence_number_outcome            : 13,642,636개 (100.00%)
patient_3_patient_weight                     : 13,642,636개 (100.00%)
patient_3_patient_sex        

In [25]:
manu_cond = True
manu_cond &= pl.col('manufacturer_g1_name') != 'None'
manu_cond &= pl.col('manufacturer_g1_name') != 'UNK'

manu_filtered = df.filter(manu_cond)

columns = manu_filtered.collect_schema().names()

# 간단하게 streaming 모드만 활성화
cols_to_keep = []
batch_size = 100

for i in range(0, len(columns), batch_size):
    batch_cols = columns[i:i + batch_size]
    
    result = manu_filtered.select([
        pl.col(col).is_not_null().any().alias(col)
        for col in batch_cols
    ]).collect(engine='streaming')
    
    cols_to_keep.extend([
        col for col in batch_cols if result[col][0]
    ])

manu_filtered = manu_filtered.select(cols_to_keep)
total_rows = manu_filtered.select(pl.len()).collect()

In [26]:
unk_lst = df.filter(pl.col('device_0_model_number').str.to_uppercase().str.contains(r'^UNK')).select('device_0_model_number').collect().to_series().to_list()
set(unk_lst)

{'UNK',
 'UNK - COOLTIP ANT',
 'UNK - I/A HANDPIECE',
 'UNK - PILLCAM',
 'UNK - PRODUCT - CORDIS CORPORATION',
 'UNK - SPECTRUM',
 'UNK - VESSEL DILATOR',
 'UNK -I/A HANDPIECE',
 'UNK 014 ALLSTAR',
 'UNK 014 BHW',
 'UNK 014 BMW HYDRO',
 'UNK 014 FLOPPY HYDRO',
 'UNK 014 HI-TORQUE PILOT',
 'UNK 014 HT TURNTRAC ES',
 'UNK 014 IRONMAN HYDRO',
 'UNK 014 WHISPER ES',
 'UNK ABSOLUTE PRO',
 'UNK ABSOLUTE PRO LL',
 'UNK ABSOLUTE PRO VASCULAR',
 'UNK ABSORB',
 'UNK ALARIS PUMP MODULE',
 'UNK AMPLATZER AMULET',
 'UNK AMPLATZER AMULET DS',
 'UNK AMPLATZER CARDIAC PLUG',
 'UNK AMPLATZER DUCT OCCL II, AS',
 'UNK AMPLATZER DUCT OCCLUDER',
 'UNK AMPLATZER DUCT OCCLUDER II',
 'UNK AMPLATZER FSO CRIBRIFORM',
 'UNK AMPLATZER OCCLUDER',
 'UNK AMPLATZER PFO OCCLUDER',
 'UNK AMPLATZER PICCOLO',
 'UNK AMPLATZER SEPTAL OCCLUDER',
 'UNK AMPLATZER TORQVUE',
 'UNK AMPLATZER VASCULAR PLUG',
 'UNK AMPLATZER VASCULAR PLUG II',
 'UNK AMPLATZER VASCULAR PLUG IV',
 'UNK AMPLATZERVASCULARPLUG III',
 'UNK ARMADA',
 'UN

In [27]:
def filter_notna_rows(df: pl.LazyFrame, col) -> Tuple[pl.LazyFrame, pl.Expr, pl.DataFrame]:
    na_patterns = r'^None$|^UNK|^NOT APPLICABLE$|^N/A$|^NA$|^$|\s+$'
    notna_cond = True
    notna_cond &= ~pl.col(col).str.to_uppercase().str.contains(na_patterns)
    filtered = df.filter(notna_cond)
    notna_rows = filtered.select(pl.len()).collect()
    return filtered, notna_cond, notna_rows

In [28]:
total_rows = df.select(pl.len()).collect().item()

udi_filtered, udi_notna_cond, udi_notna_rows = filter_notna_rows(df, 'device_0_udi_di')
model_filtered, model_notna_cond, model_notna_rows = filter_notna_rows(df, 'device_0_model_number')

coverage_analysis = df.select([
    udi_notna_cond.is_not_null().alias("has_udi"),
    model_notna_cond.is_not_null().alias("has_model")
]).group_by(["has_udi", "has_model"]).len().with_columns([
    pl.format("{}%", (pl.col("len").mul(100).truediv(total_rows).round(1)).alias("pct"))
])

# 실제 커버리지 계산
actual_coverage = df.select([
    (udi_notna_cond | model_notna_cond).mean().alias("either"),
    (udi_notna_cond & model_notna_cond).mean().alias("both")
])

coverage_analysis.collect(), actual_coverage.collect()

(shape: (4, 4)
 ┌─────────┬───────────┬─────────┬───────┐
 │ has_udi ┆ has_model ┆ len     ┆ pct   │
 │ ---     ┆ ---       ┆ ---     ┆ ---   │
 │ bool    ┆ bool      ┆ u32     ┆ str   │
 ╞═════════╪═══════════╪═════════╪═══════╡
 │ true    ┆ true      ┆ 7471374 ┆ 54.8% │
 │ false   ┆ true      ┆ 2755969 ┆ 20.2% │
 │ true    ┆ false     ┆ 1754223 ┆ 12.9% │
 │ false   ┆ false     ┆ 1661070 ┆ 12.2% │
 └─────────┴───────────┴─────────┴───────┘,
 shape: (1, 2)
 ┌────────┬──────────┐
 │ either ┆ both     │
 │ ---    ┆ ---      │
 │ f64    ┆ f64      │
 ╞════════╪══════════╡
 │ 1.0    ┆ 0.935589 │
 └────────┴──────────┘)

In [29]:
df.filter(udi_notna_cond & ~model_notna_cond).first().select('device_0_udi_di').collect()

device_0_udi_di
str
"""00389701008851"""


In [30]:
device_cols = [f'device_{i}_udi_di' for i in range(50)]
model_cols = [f'device_{i}_model_number' for i in range(50)]

target_cols = device_cols
unique_values = set()

for c in target_cols:
    lst = df.select(pl.col(c).unique())
    unique_values.update(lst.collect().to_series().to_list())

print(len(unique_values))

maude_udi = unique_values

142336


In [31]:
df.filter(pl.col('device_0_brand_name').str.to_uppercase().str.starts_with('ACCELERATOR')).collect().to_pandas()

,adverse_event_flag,date_added,date_changed,date_facility_aware,date_manufacturer_received,date_of_event,date_received,date_report,date_report_to_fda,date_report_to_manufacturer,...,report_to_manufacturer,reporter_country_code,reporter_occupation_code,reprocessed_and_reused_flag,single_use_flag,source_type,summary_report_flag,suppl_dates_fda_received,suppl_dates_mfr_received,type_of_report
0,Y,20230901,20240409,None,20230804,None,20230901,20231012,None,None,...,None,US,003,N,N,['Distributor'],N,10/12/2023,09/14/2023,"['Initial submission', 'Followup']"
1,Y,20230831,20230907,None,None,None,20230829,20230825,None,None,...,None,US,OTHER,None,None,None,N,None,None,['Initial submission']
2,N,20201210,20201224,None,20201112,20201106,20201210,20201210,None,None,...,None,US,003,N,N,['DISTRIBUTOR'],N,None,None,['Initial submission']
3,N,20201021,20230803,None,20200925,None,20201021,20201116,None,None,...,None,None,None,N,None,"['FOREIGN', 'HEALTH PROFESSIONAL']",N,11/16/2020,11/06/2020,"['Initial submission', 'Followup']"
4,N,20201016,20230808,None,20200924,None,20201016,20201105,None,None,...,None,None,None,N,None,"['FOREIGN', 'HEALTH PROFESSIONAL']",N,11/05/2020,11/05/2020,"['Initial submission', 'Followup']"
5,Y,20250903,20251021,None,20250804,20250804,20250903,20251021,None,None,...,None,US,003,N,None,['Distributor'],N,10/21/2025,09/23/2025,"['Initial submission', 'Followup']"
6,N,20211019,20231024,None,20210929,20210929,20211019,20211213,None,None,...,None,CA,OTHER HEALTH CARE PROFESSIONAL,N,N,"['Foreign', 'Health Professional']",N,12/13/2021,11/30/2021,"['Initial submission', 'Followup']"
7,N,20211122,20231130,None,20211027,20211027,20211122,20220606,None,None,...,None,US,OTHER HEALTH CARE PROFESSIONAL,N,N,['Health Professional'],N,05/04/2022;06/06/2022,04/07/2021;06/06/2022,"['Initial submission', 'Followup', 'Followup']"
8,N,20200416,20200417,None,20200326,20200326,20200416,20200416,None,None,...,None,US,OTHER,N,N,['COMPANY REPRESENTATIVE'],N,None,None,['Initial submission']
9,N,20200413,20230531,None,20200325,None,20200413,20200512,None,None,...,None,None,None,N,None,"['FOREIGN', 'HEALTH PROFESSIONAL']",N,05/12/2020,04/30/2020,"['Initial submission', 'Followup']"


In [32]:
df.select(pl.col('patient_0_patient_sequence_number', 'patient_1_patient_sequence_number', 'patient_2_patient_sequence_number', 'patient_3_patient_sequence_number').is_not_null().sum()).collect()

patient_0_patient_sequence_number,patient_1_patient_sequence_number,patient_2_patient_sequence_number,patient_3_patient_sequence_number
u32,u32,u32,u32
13637345,117,2,1


In [33]:
total_rows = df.select(pl.len()).collect()

manu_col = 'device_0_manufacturer_d_name'
udi_col = 'device_0_udi_di'
model_col = 'device_0_model_number'
manu_filtered, manu_notna_cond, manu_notna_rows = filter_notna_rows(df, manu_col)
udi_filtered, udi_notna_cond, udi_notna_rows = filter_notna_rows(df, udi_col)
model_filtered, model_notna_cond, model_notna_rows = filter_notna_rows(df, model_col)

print(manu_notna_rows / total_rows * 100)
print(udi_notna_rows / total_rows * 100)
print(model_notna_rows / total_rows * 100)

shape: (1, 1)
┌──────────┐
│ len      │
│ ---      │
│ f64      │
╞══════════╡
│ 99.38152 │
└──────────┘
shape: (1, 1)
┌───────────┐
│ len       │
│ ---       │
│ f64       │
╞═══════════╡
│ 67.623273 │
└───────────┘
shape: (1, 1)
┌───────────┐
│ len       │
│ ---       │
│ f64       │
╞═══════════╡
│ 71.288855 │
└───────────┘


In [34]:
total_rows = manu_filtered.select(pl.len()).collect()
manu_filtered.select('device_0_udi_di').null_count().collect() / total_rows

device_0_udi_di
f64
0.322627


In [35]:
columns = manu_filtered.collect_schema().names()
pprint(columns)

['adverse_event_flag',
 'date_added',
 'date_changed',
 'date_facility_aware',
 'date_manufacturer_received',
 'date_of_event',
 'date_received',
 'date_report',
 'date_report_to_fda',
 'date_report_to_manufacturer',
 'device',
 'device_0_brand_name',
 'device_0_catalog_number',
 'device_0_combination_product_flag',
 'device_0_date_received',
 'device_0_date_removed_flag',
 'device_0_date_returned_to_manufacturer',
 'device_0_device_age_text',
 'device_0_device_availability',
 'device_0_device_evaluated_by_manufacturer',
 'device_0_device_event_key',
 'device_0_device_operator',
 'device_0_device_report_product_code',
 'device_0_device_sequence_number',
 'device_0_expiration_date_of_device',
 'device_0_generic_name',
 'device_0_implant_flag',
 'device_0_lot_number',
 'device_0_manufacturer_d_address_1',
 'device_0_manufacturer_d_address_2',
 'device_0_manufacturer_d_city',
 'device_0_manufacturer_d_country',
 'device_0_manufacturer_d_name',
 'device_0_manufacturer_d_postal_code',
 'dev

In [36]:

mdr_cond = pl.col('mdr_text_79_text') != 'None'
device_cond = pl.col('device_47_device_report_product_code') != 'None'
patient_cond = pl.col('patient_3_patient_sequence_number') != 'None'

pprint(manu_filtered.filter(device_cond).collect().to_dicts())
# df.filter(pl.col('device_54_device_report_product_code') != 'None').collect()
# df.filter(pl.col('patient_3_patient_sequence_number') != 'None').collect()

[{'adverse_event_flag': 'Y',
  'date_added': '20221222',
  'date_changed': '20230125',
  'date_facility_aware': None,
  'date_manufacturer_received': None,
  'date_of_event': None,
  'date_received': '20221219',
  'date_report': '20221215',
  'date_report_to_fda': None,
  'date_report_to_manufacturer': None,
  'device': None,
  'device_0_brand_name': 'CASSETTE MEDICATION RESERVOIR',
  'device_0_catalog_number': None,
  'device_0_combination_product_flag': 'Y',
  'device_0_date_received': '20221219',
  'device_0_date_removed_flag': None,
  'device_0_date_returned_to_manufacturer': None,
  'device_0_device_age_text': None,
  'device_0_device_availability': 'No answer provided',
  'device_0_device_evaluated_by_manufacturer': None,
  'device_0_device_event_key': None,
  'device_0_device_operator': 'LAY USER/PATIENT',
  'device_0_device_report_product_code': 'LHI',
  'device_0_device_sequence_number': '1',
  'device_0_expiration_date_of_device': None,
  'device_0_generic_name': 'SET, I.V. F

## udi 탐색

In [37]:
udi_loader = DataLoader(
    name='udi',
    start=None,
    end=None,
    output_file=DATA_DIR/'udi.parquet',
)

udi_lf = udi_loader.load(adapter, **polars_kwargs)
udi_lf


📖 /home/dataiku/eric/Sparta/Project3/data/udi.parquet 로딩 중... (adapter=polars)


In [38]:
from code.preprocess.preprocess import get_pattern_cols

udi_pattern = [r'^identifiers_.*_id$']

In [39]:
udi_cols = get_pattern_cols(udi_lf, udi_pattern)
len(udi_cols)

264

In [40]:
# 스트리밍 + unpivot 조합 (대용량 데이터에 최적)
total_udi = set(
    udi_lf.select(udi_cols).unpivot(on=udi_cols)
    .select('value')
    .unique()
    .collect(engine='streaming')
    ['value']
)

In [41]:
len(maude_udi)

142336

In [42]:
len(total_udi & maude_udi)

111611

In [45]:
unk_udi = maude_udi - total_udi
len(unk_udi)

30725

In [46]:
sorted(unk_udi)[:100]

['(01)(10)NA',
 '(01)SW11677',
 '(01)SW12300',
 '(01)UNKNOWN',
 '+B095321804',
 '+D590135020',
 '+D950135020',
 '+M20656410',
 '+M206715716',
 '+M20674440',
 '+M20674441',
 '+M724BSMM1',
 '+M724CCB541',
 '+M724CCB551',
 '+M724MPJS1',
 '0',
 '0(01)(10)93141',
 '0(01)(21)11442',
 '0(01)(21)70620',
 '0(01)008848380',
 '0(01)100302881',
 '0(01)26202031B',
 '0+$$+75299/16D',
 '0+D66160013941',
 '0+D661H0082019',
 '0+H20282316411',
 '0+H66860517001',
 '0+M57250600020',
 '0+M724CCB6T721',
 '00',
 '00(01)(10)5121',
 '00(01)(17)(10)',
 '00(01)(21)3051',
 '00(01)(21)3088',
 '00(01)(21)3150',
 '00(01)(21)3168',
 '00(01)(21)3170',
 '00(01)(21)3206',
 '00(01)(21)3254',
 '00(01)(21)3270',
 '00(01)(21)3306',
 '00(01)(21)3313',
 '00(01)(21)3323',
 '00(01)(21)3330',
 '00(01)(21)3333',
 '00(01)(21)3339',
 '00(01)(21)3365',
 '00(01)(21)3370',
 '00(01)(21)3401',
 '00(01)(21)3418',
 '00(01)(21)3454',
 '00(01)(21)3459',
 '00(01)(21)3485',
 '00(01)(21)3499',
 '00(01)(21)3503',
 '00(01)(21)3552',
 '00(01)(21)